In [177]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import seaborn as sns
import matplotlib as mpl
mpl.rc("font", family="Malgun Gothic")

- csv 파일은 read_csv로
- xlsx 파일은 read_excel로 불러온다.
- col이름이 이상하다.
> - 구분, 카테고리명, 자재그룹...은 항목명으로 만들어야 한다.
> - 날짜값들은 새로운 col 만들어서 정리해야 한다. 
> - 그래야 분석 및 시각화가 가능하다.
> -정상, 불용.. 제외하고 불러오자.

- pd.read_excel(skiprows)
> 행 생갹하고 그 뒤 행부터 컬럼명으로 잡아준다. (불필요한 행 제거)

- df2 = df1.drop(columns=cols)
- df2.melt(id_vars)
> id_vars를 기준으로
col에 있는 나머지 값들을 하나의 항목으로 만들고,
그리고 그 안에 구성되어 있던 모든 data들을
value라는 col로 보내버린다.
데이터를 이렇게 바꿔준다.
날짜별, 판매제품과 매출액이 계산됬다.

- 이런 식으로 회사 erp에 있는 복잡하고 이상한 데이터를 `melt`를 사용해서 깔끔하게 정리할 수 있다. 


1. 


2. 

In [122]:
df = pd.read_excel("dataset/05_Stack.xlsx", skiprows=1)

In [124]:
df.columns

Index([                   '구분',                 '카테고리명',
                        '자재그룹',                 '자재그룹명',
                         '제품명',                    '단위',
                          '판매',     2019-12-01 00:00:00,
           2020-10-01 00:00:00,     2020-11-01 00:00:00,
           2020-12-01 00:00:00, '2019-12-01 00:00:00.1',
       '2020-10-01 00:00:00.1', '2020-11-01 00:00:00.1',
       '2020-12-01 00:00:00.1', '2019-12-01 00:00:00.2',
       '2020-10-01 00:00:00.2', '2020-11-01 00:00:00.2',
       '2020-12-01 00:00:00.2', '2019-12-01 00:00:00.3',
       '2020-10-01 00:00:00.3', '2020-11-01 00:00:00.3',
       '2020-12-01 00:00:00.3',                  '안전재고',
                         ' 분류'],
      dtype='object')

## 1
[제품명, 날짜] col만 남긴 `df` 사용

In [134]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic")

cols = ['구분','카테고리명','자재그룹',
        '자재그룹명','단위','판매', '안전재고', ' 분류']


df1 = df.drop(columns=cols).melt(id_vars="제품명").rename(columns={"variable": "일자", "value":"매출액"})
#sns.barplot(data=df1, x="제품명", y="매출액", errorbar=None)

C:\Users\USER\AppData\Local\Temp\ipykernel_15220\3700730197.py:9: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df1 = df.drop(columns=cols).melt(id_vars="제품명").rename(columns={"variable": "일자", "value":"매출액"})


## 2
[날짜] 제외한 모든 col을 `melt(id_vars)`에 집어넣은 `df` 사용

In [176]:
cols = ['구분','카테고리명', '자재그룹','자재그룹명',
        '제품명','단위', '판매', '안전재고', ' 분류']
df2 = df.melt(id_vars=cols).rename(columns={"variable": "일자", "value":"매출액"})
df2["연도"] = df2["일자"].dt.year
df2["월"] = df2["일자"].dt.month

#sns.barplot(data=df2, x="월", y="매출액", hue="연도", errorbar=None) # 19년도 12월 데이터만 있다. 

# 12월만 filtering
cond1 = (df2["월"] == 12)
df3 = df2.loc[cond1]

# 제품에 따른 연도별 12월 매출액 sum
df4 = df3.pivot_table(index="제품명", columns="연도", values="매출액", aggfunc="sum")

# 증감액 & 증감률 산출
df4["증감액"] = df4[2020] - df4[2019]
df4["증감률"] = df4["증감액"] / (df4[2020] + df4[2019])


# 작년 and 올해 모두 판매한 제품의 매출액 증감률
## 증감률 1: 전년도 매출액이 없는 제품의 매출액 증감률
## 증감률 -1.0: 작년까지만 판매하고 올해는 판매하지 않은 제품의 매출액 증감률
## 따라서 증감률이 1.00 미만인 제품만 본다. (작년 and 올해 판매한 제품의 증감률만 의미 있음)
cond1 = (df4["증감률"] > -1.0) & (df4["증감률"] < 1.0)
df5 = df4.loc[cond1]


# M, B 제품은 전년도 대비 증감률이 90% 이상
df5.sort_values(by="증감률", ascending=False).head()

C:\Users\USER\AppData\Local\Temp\ipykernel_15220\3061482346.py:3: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df2 = df.melt(id_vars=cols).rename(columns={"variable": "일자", "value":"매출액"})


연도,2019,2020,증감액,증감률
제품명,,,,
M,28.0,52995.0,52967.0,0.998944
B2,1039.0,23212.6,22173.6,0.914315
F,6363.6,16589.8,10226.2,0.445520
H,52483.6,121474.0,68990.4,0.396593
A2,36546.0,68027.4,31481.4,0.301046
